# Kvantni algoritmi za aproksimativnu optimizaciju (QAOA)

Jedna bitna klasa kvantnih algoritama su QAOA algoritmi. Ovde ćemo dati teorijske osnove ovakvih algoritama.

Glavni problem koji QAOA rešava je nalaženje minimuma neke funkcije $C$ koja zavisi od instance nekog problema. Na primer, problem može biti nalaženje najkraćeg puta između dve tačke u gradu, instanca problema taj grad, a funkcija koju želimo da minimizujemo je $C(\pi_{A, B}) = \text{length}(\pi_{A, B})$, gde je $\pi_{A, B}$ neki put od tačke $A$ do tačke $B$.

Ovakve probleme je često jako teško rešiti egzaktno i zato postoje neke aproksimativne šeme koje nam daju garanciju da naše rešenje, koje nalazimo za dosta manje vremena, neće biti mnogo gore od onog koje je zapravo najbolje. Formalno, neka je $m$ minimum funkcije $C$, a $m'$ vrednost koju smo mi dobili. Uvodimo konstantu $\alpha$ takvu da je:
\begin{align*}
    m' \leq \alpha m,
\end{align*} za sve instance problema.

Koliko je dobra aproksimacija se procenjuje na osnovu toga koliko je $\alpha$ blizu $1$.

QAOA se često naziva diskretizovanim adijabatskim računanjem. Osnovna ideja algoritma je da pretvori funkciju cene u neku ermitsku matricu koju zovemo *Hamiltonijan cene*, ili samo *Hamiltonijan*. Obično se to realizuje mapiranjem u težinsku sumu Paulijevih stringova, što olakšava dalje računanje. Za ovaj Hamiltonijan je potrebno da važi da neka sopstvena vrednost (poznato nam je koja po redu) odgovara minimumu funkcije $C$. Nakon toga, definiše se drugi Hamiltonijan, koji se zove *mikser Hamiltonijan*, ili samo *mikser*, čija je odgovarajuća sopstvena vrednost laka za računanje i čiji sopstveni vektor možemo lako da instanciramo na kvantnom računaru. Adijabatskim računanjem bismo sada instancirali taj vektor u mikseru, pa bismo ga *napadali* operatorima evolucije, dok ne dođe do sopstvenog vektora od Hamiltonijana cene koji odgovara minimumu funkcije. QAOA to radi malo drugačije: definiše se ansatz, kao što se radi u algoritmu VQE, a zatim se njegovi parametri poboljšavaju dok se dobije najbolji opis vremenske evolucije koji će prevesti mikserovo stanje u stanje minimuma.

Sumirajmo korake:
1. transformiši funkciju $C$ u ermitsku matricu $H_1$ (Hamiltonijan cene) takvu da je neka (najčešće najmanja) sopstvena vrednost te matrice minimum funkcije $C$
2. konstruiši ermitsku matricu $H_0$ (mikser Hamiltonijan) čija je odgovarajuća (najčešće najmanja) sopstvena vrednost laka za računanje i odgovarajući sopstveni vektor može brzo da se instancira na kvantnom računaru;
3. konstruiši ansatz, parametrizovano kvantno kolo koje opisuje evoluciju sistema iz $H_0$ u $H_1$;
4. izaberi funkciju $F$ koju ćeš minimizovati: najčešće je to neka funkcija koja meri grešku aproksimacije kada se ansatz primeni na sopstveni vektor miksera; ta funkcija će nam reći koliko smo blizu stanju koje tražimo;
5. izaberi omiljeni algoritam optimizacije;
6. izaberi početne parametre za ansatz
7. u petlji, dok optimizator ne reši da stane:
   1. koristi ansatz i parametre da sračunaš $F$
   2. na osnovu vrednosti $F$, odredi nove parametre za ansatz
8. na osnovu parametara koje dobiješ na kraju optimizacije, sračunaj vrednost ansatza tu i nađi odgovarajuću sopstvenu vrednost matrice $H_1$, odnosno minimum funkcije $C$.

## Primer:

Data je funkcija $f(x) = x^T Q x$ za vektor $x$ dužine $n$ i simetričnu matricu $Q$ $n \times n$.

Smenom promenljivih $x_i = \frac{1 - z_i}{2}$, dobijamo:
\begin{align*}
    x^T Q x &= \sum_{i = 1}^{n}\sum_{j = 1}^{n} Q_{i,j} x_i x_j, \\
            &= \frac{1}{4} \sum_{i = 1}^{n}\sum_{j = 1}^{n} Q_{i, j}(1 - z_i)(1 - z_j), \\
            &= \frac{1}{4} \sum_{i = 1}^{n}\sum_{j = 1}^{n} Q_{i, j} z_i z_j - \frac{1}{4} \sum_{i = 1}^{n}\sum_{j = 1}^{n} (Q_{i, j} + Q_{j, i}) z_i + \frac{n^2}{4}.
\end{align*}

Postavljanjem $b_i = \sum{j = 1}^{n} (Q_{i, j} + Q_{j, i})$, dobijamo:
\begin{align*}
    4 x^T Q x &= z^T Q z + b^T z + n^2.
\end{align*}

Iz ovoga sledi da je dovoljno naći $\min_{z_i \in \{-1, 1\}}(z^T Q z + b^T z)$ da bi se odredio minimum funkcije $f$.

Odgovarajući Hamiltonijan cene je ovde:
\begin{align*}
    H_1 = \sum_{i = 1}^{n} \sum_{j = 0}^{n} Q_{i, j} Z_i Z_j + \sum_{i = 1}^{n} b_i Z_i.
\end{align*}

$Z_i$ ovde odgovara Paulijevom stringu $I \otimes \ldots \otimes Z_i \otimes \ldots \otimes I$.

Najmanja sopstvena vrednost Hamiltonijana $H_1$ je minimum funkcije $g(z) = z^T Q z + b^T z$.

Za mikser Hamiltonijan se može uzeti:
\begin{align*}
    H_0 = \sum_{i = 1}^{n} X_i.
\end{align*}

Stanje koje odgovara najmanjoj sopstvenoj vrednosti u $H_0$ je $\ket{\psi_0} = H^{\otimes n} \ket{0}^{\otimes n}$.

Ansatz će nam biti:
\begin{align*}
    \ket{\psi(\theta)} = e^{- i \gamma_p H_1} e^{-i \beta_p H_0} \ldots e^{-i \gamma_1 H_1} e^{-i \beta_1 H_0} \ket{\psi_0}.
\end{align*}, gde je parametar $p$ unapred izabran i on određuje dubinu ovog kvantnog kola. Parametar $\theta$ je vektor parametara $\gamma_i$ i $\beta_i$.

Funkcija koju minimizuje optimizator je $\bra{\psi(\theta)} H_1 \ket{\psi(\theta)}$, koja određuje vrednost najmanje sopstvene vrednosti Hamiltonijana $H_1$.